In [3]:
# !pip install torch

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open("rocities.txt", "r").read().splitlines()
words[:10]

['1 DECEMBRIE',
 '2 MAI',
 '23 AUGUST',
 'ABRAM',
 'ABRAMUT',
 'ABRUD',
 'ABRUD-SAT',
 'ABUCEA',
 'ABUD',
 'ABUS']

In [4]:
len(words)

10148

In [8]:
# build the vocabulary and mappings to/from ints
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i,s in enumerate(chars)}
stoi['*'] = 0
itos = {i: s for s,i in stoi.items()}
print(stoi)
chars_count = len(stoi.values())

{' ': 1, '-': 2, '.': 3, '1': 4, '2': 5, '3': 6, 'A': 7, 'B': 8, 'C': 9, 'D': 10, 'E': 11, 'F': 12, 'G': 13, 'H': 14, 'I': 15, 'J': 16, 'K': 17, 'L': 18, 'M': 19, 'N': 20, 'O': 21, 'P': 22, 'R': 23, 'S': 24, 'T': 25, 'U': 26, 'V': 27, 'X': 28, 'Y': 29, 'Z': 30, '*': 0}


In [9]:
# dataset

block_size = 3 # length of context: no. of characters we use to predict the next

X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '*':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '-->', itos[ix])
        context = context[1:] + [ix] # crop and append, like a rolling window
        
X = torch.tensor(X)
Y = torch.tensor(Y)

1 DECEMBRIE
*** --> 1
**1 -->  
*1  --> D
1 D --> E
 DE --> C
DEC --> E
ECE --> M
CEM --> B
EMB --> R
MBR --> I
BRI --> E
RIE --> *
2 MAI
*** --> 2
**2 -->  
*2  --> M
2 M --> A
 MA --> I
MAI --> *
23 AUGUST
*** --> 2
**2 --> 3
*23 -->  
23  --> A
3 A --> U
 AU --> G
AUG --> U
UGU --> S
GUS --> T
UST --> *
ABRAM
*** --> A
**A --> B
*AB --> R
ABR --> A
BRA --> M
RAM --> *
ABRAMUT
*** --> A
**A --> B
*AB --> R
ABR --> A
BRA --> M
RAM --> U
AMU --> T
MUT --> *


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([42, 3]), torch.int64, torch.Size([42]), torch.int64)

In [10]:
chars_count

31

In [11]:
C = torch.randn((chars_count, 2))  # lookup table - each char is 2 dimensional

In [12]:
C

tensor([[-1.1380, -0.2441],
        [-1.4069, -2.0488],
        [ 0.7973,  2.0267],
        [-0.1058,  0.9866],
        [-0.2701,  1.3753],
        [ 0.7238,  0.3643],
        [-1.2547,  1.1373],
        [ 0.7344, -0.8236],
        [-0.3595,  0.4817],
        [ 0.3307,  0.6951],
        [ 0.0636, -1.1992],
        [ 0.1435,  0.7069],
        [-1.5492, -0.3226],
        [ 0.4631, -0.2023],
        [-1.3454, -1.5485],
        [ 0.4418, -0.7414],
        [-1.8547, -0.9415],
        [ 0.8961,  0.4491],
        [-0.7651,  0.6367],
        [-1.4863,  1.1257],
        [-0.1860, -0.0209],
        [ 1.0618, -1.0539],
        [ 0.2980, -0.0723],
        [ 0.1165,  0.0628],
        [ 0.2156, -0.5296],
        [-0.2141, -0.4940],
        [-0.1915,  0.7709],
        [ 2.0043,  0.0134],
        [-1.9098,  1.6056],
        [-1.4830,  1.9992],
        [ 0.9184,  0.0464]])